## Python file to clean BQ DB

In [1]:
#Import des librairies nécessaires
import pandas as pd
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
# Inspecation des données 
QUERY = """
SELECT *
FROM `dash-vlille.data_table_vlille_59000.table_2025`
WHERE is_modified IS NULL
LIMIT 100
"""
query_job = client.query(QUERY)
results = query_job.result()
df = results.to_dataframe()

c:\Users\coulo\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
df

,ID_station,geometry_coordinates,properties_nom,properties_adresse,properties_commune,properties_etat,properties_nb_places_dispo,properties_nb_velos_dispo,properties_etat_connexion,properties_x,properties_y,properties_date_modification,Date_Scrapping,Heure_Min_Scrapping,Date_Paris,is_modified
0,vlille_temps_reel.1,"[3.075992, 50.641926]",METROPOLE EUROPEENNE DE LILLE,MEL RUE DU BALLON,LILLE,RÉFORMÉ,0,0,DÉCONNECTÉ,3.075992,50.641926,2022-11-29T10:47:16.181+00:00,14/11/2025,21:00,NaT,<NA>
1,vlille_temps_reel.105,"[3.10231, 50.638126]",VERDUN,69 AVENUE DE VERDUN,MONS EN BAROEUL,RÉFORMÉ,0,0,DÉCONNECTÉ,3.102310,50.638126,2022-11-29T10:47:16.183+00:00,14/11/2025,21:00,NaT,<NA>
2,vlille_temps_reel.106,"[3.112704, 50.638123]",CONCORDE,27 BOULEVARD ALPHONSE GAYET,MONS EN BAROEUL,RÉFORMÉ,0,0,DÉCONNECTÉ,3.112704,50.638123,2022-11-29T10:47:16.183+00:00,14/11/2025,21:00,NaT,<NA>
3,vlille_temps_reel.109,"[3.030667, 50.623466]",SQUARE DU PORTUGAL,FACE AU 204 RUE DE LA BASSEE,LILLE,RÉFORMÉ,0,0,DÉCONNECTÉ,3.030667,50.623466,2022-11-29T10:47:16.184+00:00,14/11/2025,21:00,NaT,<NA>
4,vlille_temps_reel.112,"[3.115101, 50.626846]",SQUARE CHARLES DE GAULLE,"4, RUE FERDINAND BUISSON",HELLEMMES,RÉFORMÉ,0,0,DÉCONNECTÉ,3.115101,50.626846,2022-11-29T10:47:16.184+00:00,14/11/2025,21:00,NaT,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,vlille_temps_reel.81,"[3.115088, 50.63529]",GUINGUETTE,208 rue Corneille,Hellemmes,EN SERVICE,7,3,CONNECTÉ,3.115088,50.635290,2025-11-14T21:59:09.522+00:00,14/11/2025,21:00,NaT,<NA>
96,vlille_temps_reel.85,"[3.1008, 50.62504]",CHANZY,102 rue Chanzy,Hellemmes,EN SERVICE,9,3,CONNECTÉ,3.100800,50.625040,2025-11-14T21:59:09.523+00:00,14/11/2025,21:00,NaT,<NA>
97,vlille_temps_reel.86,"[3.10295, 50.62903]",ROGER SALENGRO,58 rue Roger Salengro,Hellemmes,EN SERVICE,9,3,CONNECTÉ,3.102950,50.629030,2025-11-14T21:59:09.523+00:00,14/11/2025,21:00,NaT,<NA>
98,vlille_temps_reel.88,"[3.058015, 50.635082]",RUE NATIONALE,89 rue Nationale,Lille,EN SERVICE,27,3,CONNECTÉ,3.058015,50.635082,2025-11-14T21:59:09.524+00:00,14/11/2025,21:00,NaT,<NA>


In [4]:
df_copy = df.copy(deep=True)

In [ ]:
# Nettoyage et transformation des données
#Ajout d'une colonne pour marquer les lignes modifiées
#Ajout de la colonne Date_Paris
#Cleaning des dates et retrait de texte inutile
QUERY = """
-- Ajoute une colonne Date_Paris au format DATETIME
# ALTER TABLE `dash-vlille.data_table_vlille_59000.table_2025`
# ADD COLUMN Date_Paris DATETIME,
# ADD COLUMN is_modified BOOL;

-- Met à jour la colonne avec la conversion à l'heure de Paris
UPDATE `dash-vlille.data_table_vlille_59000.table_2025`

SET Date_Paris = DATETIME(TIMESTAMP(properties_date_modification), "Europe/Paris"),
    Date_Scrapping = FORMAT_DATE("%d-%m-%Y", PARSE_DATE("%d/%m/%Y", REPLACE(Date_Scrapping, '_', '/'))),
    ID_station = REPLACE(ID_station, "vlille_temps_reel.", ""),
    Heure_Min_Scrapping = FORMAT_TIME("%H:%M", PARSE_TIME("%H:%M", REPLACE(REPLACE(Heure_Min_Scrapping, "_", ":"),".c", ""))),
    is_modified = TRUE
WHERE is_modified IS NULL;
"""
query_job = client.query(QUERY)
results = query_job.result()
df = results.to_dataframe()


In [20]:
QUERY = """
SELECT *
FROM `dash-vlille.data_table_vlille_59000.table_2025`
WHERE IS_MODIFIED = TRUE
LIMIT 10
"""
query_job = client.query(QUERY)
results = query_job.result()
df_modified = results.to_dataframe()


c:\Users\coulo\VLille\V-Lille-Dash\lab\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [21]:
df_modified.head()

,ID_station,geometry_coordinates,properties_nom,properties_adresse,properties_commune,properties_etat,properties_nb_places_dispo,properties_nb_velos_dispo,properties_etat_connexion,properties_x,properties_y,properties_date_modification,Date_Scrapping,Heure_Min_Scrapping,Date_Paris,is_modified
0,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T09:25:17.446+00:00,14-03-2025,08:26,2025-03-14 10:25:17.446,True
1,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-11-05T08:45:11.429+00:00,05-11-2025,07:45,2025-11-05 09:45:11.429,True
2,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T09:41:18.075+00:00,14-03-2025,08:42,2025-03-14 10:41:18.075,True
3,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T11:16:16.070+00:00,14-03-2025,10:16,2025-03-14 12:16:16.070,True
4,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T22:15:19.907+00:00,14-03-2025,21:15,2025-03-14 23:15:19.907,True
